# Generate Test Questions

In [1]:
from training.training_data_generator import TrainingDataGenerator

# Initialize training data generator
generator = TrainingDataGenerator(
    question_bank_path='training/question_bank.md', 
    trees_csv_path='data/trees_df.csv', 
    patches_csv_path='data/patches_df.csv'
)

# Generate training data
df = generator.generate_dataset()
print(f"Generated {len(df)} training examples")
print(f"\n{df['intent'].value_counts()}")

# Extract test questions
test_questions = df['text']

Generated 576 training examples

intent
patch_requirements      108
tree_requirements       104
tree_recommendations     91
payment                  78
growth_time              65
transportation           60
list_regular_patches     36
list_fruit_patches       34
Name: count, dtype: int64


# Test Intent Classifier

In [2]:
from source.intent_classifier import IntentClassifier

# Initialize intent classifier
classifier = IntentClassifier()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    print(f'Question: {question}')
    print(f'Intent: {intent}')
    print(f'Confidence: {confidence}')
    print('')

Question: What quest is needed for the Catherby patch?
Intent: patch_requirements
Confidence: 0.7449981016372944

Question: What level is needed to grow Papaya trees?
Intent: tree_requirements
Confidence: 0.7111690821270885

Question: How do I protect Apple trees?
Intent: payment
Confidence: 0.6431647931658729

Question: What is the nearest teleport to the Varrock patch?
Intent: transportation
Confidence: 0.7623833104212006

Question: How long does Papaya take to grow?
Intent: growth_time
Confidence: 0.8223888355284071

Question: How long does Dragonfruit take to grow?
Intent: growth_time
Confidence: 0.8042654461863591

Question: What quest is needed for the Falador patch?
Intent: patch_requirements
Confidence: 0.7944659887499776

Question: Which are the best trees at level 27?
Intent: tree_recommendations
Confidence: 0.7836049769245355

Question: Which are the best trees at level 75?
Intent: tree_recommendations
Confidence: 0.7925483473561532

Question: What do I pay the farmer to pro

# Test Answer Retriever

In [3]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Retrieve answer for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    result = retriever.get_answer(question, intent)
    print(f'Question: {question}')
    print(f'Intent: {intent}')
    if intent == 'list_regular_patches' or intent == 'list_fruit_patches':
        print(f'Answer: {", ".join([p["location"] for p in result["patches"]])}')
    elif intent == 'tree_recommendations':
        print(f'Lookup: {result["lookup_value"]}')
        print(f'Regular Tree: {result["regular_tree"]}')
        print(f'Fruit Tree: {result["fruit_tree"]}')
    else:
        print(f'Lookup: {result["lookup_value"]}')
        print(f'Answer: {result["answer_value"]}')
    print('')

Question: What quest is needed for the Catherby patch?
Intent: patch_requirements
Lookup: Catherby
Answer: nan

Question: What level is needed to grow Papaya trees?
Intent: tree_requirements
Lookup: Papaya
Answer: 57

Question: How do I protect Apple trees?
Intent: payment
Lookup: Apple
Answer: 9 sweetcorn

Question: What is the nearest teleport to the Varrock patch?
Intent: transportation
Lookup: Varrock
Answer: Varrock teleport

Question: How long does Papaya take to grow?
Intent: growth_time
Lookup: Papaya
Answer: 16 hours

Question: How long does Dragonfruit take to grow?
Intent: growth_time
Lookup: Dragonfruit
Answer: 16 hours

Question: What quest is needed for the Falador patch?
Intent: patch_requirements
Lookup: Falador
Answer: nan

Question: Which are the best trees at level 27?
Intent: tree_recommendations
Lookup: 27
Regular Tree: Oak
Fruit Tree: Apple

Question: Which are the best trees at level 75?
Intent: tree_recommendations
Lookup: 75
Regular Tree: Magic
Fruit Tree: Palm

# Test Response Generator

In [4]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever
from source.response_generator import ResponseGenerator

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Initialize response generator
generator = ResponseGenerator()

# Create response for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    result = retriever.get_answer(question, intent)
    response = generator.generate_response(intent, result)
    print(f'Question: {question}')
    print(f'Response: {response}')
    print('')

Question: What quest is needed for the Catherby patch?
Response: The Catherby fruit tree patch has no requirements.

Question: What level is needed to grow Papaya trees?
Response: You can grow Papaya trees at level 57 Farming.

Question: How do I protect Apple trees?
Response: Farmers require 9 sweetcorn to protect Apple trees.

Question: What is the nearest teleport to the Varrock patch?
Response: The Varrock patch can be found in the castle courtyard.
To get there, use a Varrock teleport.

Question: How long does Papaya take to grow?
Response: Papaya trees take 16 hours to grow.

Question: How long does Dragonfruit take to grow?
Response: Dragonfruit trees will be fully grown 16 hours after planting.

Question: What quest is needed for the Falador patch?
Response: The Falador patch has no requirements.

Question: Which are the best trees at level 27?
Response: The best trees to grow at level 27 Farming are Oak and Apple.

Question: Which are the best trees at level 75?
Response: The 

# Test Orchestrator

In [1]:
from source.orchestrator import Bolongo

# Initialize Bolongo
bolongo = Bolongo()

# Ask a question
bolongo.ask("Can I eat Maple trees?")

'I think you might have had one too many Blurberry specials.'